<a href="https://colab.research.google.com/github/FranciscoMedellin/MNA_NLP/blob/main/Semana7_8/Diego_MNA_NLP_actividad_semanas_7_y_8_LDA_y_LLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Procesamiento de Lenguaje Natural**

## Maestría en Inteligencia Artificial Aplicada
#### Tecnológico de Monterrey
#### Prof Luis Eduardo Falcón Morales

### **Adtividad en Equipos Semanas 7 y 8 : LDA y LMM audio-a-texto**

* **Nombres y matrículas:**

  *   Elemento de lista
  *   Elemento de lista
  *   Elemento de lista

* **Número de Equipo:**


* ##### **En cada ejercicio pueden importar los paquetes o librerías que requieran.**

* ##### **En cada ejercicio pueden incluir las celdas y líneas de código que deseen.**

In [ ]:
import requests
import os
from urllib.parse import urlparse
import json


from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC
import librosa
import torch

from google.colab import userdata
from huggingface_hub import login
login(userdata.get('HF_TOKEN'))

from google.colab import userdata
from openai import OpenAI
import openai

api_key = userdata.get("OpenAI_dbernal")

if not api_key:
  raise ValueError("API key no encontrada en los secretos")

# Crear cliente con tu clave de API
client = OpenAI(api_key=api_key)

import re
import nltk
import string
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

nltk.download('stopwords')
from nltk.corpus import stopwords

# 📌 Stopwords en español
stop_words = set(stopwords.words('spanish'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Verificamos si hay GPU disponible:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Usando dispositivo: {device}")


Usando dispositivo: cpu


# **Ejercicio 1:**

* #### **Liga de los audios de las fábulas de Esopo:** https://www.gutenberg.org/ebooks/21144

* #### **Descargar los 10 archivos de audio solicitados: 1, 4, 5, 6, 14, 22, 24, 25, 26, 27.**



In [ ]:
# Incluyan a continuación todas las celdas (de código o texto) que deseen...


def descargar_mp3(url, carpeta_destino="archivos"):


  try:
    # Crear carpeta si no existe
    if not os.path.exists(carpeta_destino):
        os.makedirs(carpeta_destino)

    # Obtener nombre del archivo de la URL
    parsed_url = urlparse(url)
    nombre_archivo = os.path.basename(parsed_url.path)

    # Si no tiene extensión, agregar .mp3
    if not nombre_archivo.endswith('.mp3'):
        nombre_archivo += '.mp3'

    ruta_completa = os.path.join(carpeta_destino, nombre_archivo)

    # Descargar el archivo
    print(f"Descargando: {nombre_archivo}")
    response = requests.get(url, stream=True)
    response.raise_for_status()

    with open(ruta_completa, 'wb') as f:
        for chunk in response.iter_content(chunk_size=8192):
            f.write(chunk)

    print(f"✓ Descargado: {ruta_completa}")
    return True

  except Exception as e:
    print(f"Error descargando {url}: {str(e)}")
    return False

# Ejemplo de uso
audio_solicitados = [1, 4, 5, 6, 14, 22, 24, 25, 26, 27]
urls_mp3 = [f"https://www.gutenberg.org/files/21144/mp3/21144-{x:02d}.mp3" for x in audio_solicitados]

for url in urls_mp3:
    descargar_mp3(url)

Descargando: 21144-01.mp3
✓ Descargado: archivos/21144-01.mp3
Descargando: 21144-04.mp3
✓ Descargado: archivos/21144-04.mp3
Descargando: 21144-05.mp3
✓ Descargado: archivos/21144-05.mp3
Descargando: 21144-06.mp3
✓ Descargado: archivos/21144-06.mp3
Descargando: 21144-14.mp3
✓ Descargado: archivos/21144-14.mp3
Descargando: 21144-22.mp3
✓ Descargado: archivos/21144-22.mp3
Descargando: 21144-24.mp3
✓ Descargado: archivos/21144-24.mp3
Descargando: 21144-25.mp3
✓ Descargado: archivos/21144-25.mp3
Descargando: 21144-26.mp3
✓ Descargado: archivos/21144-26.mp3
Descargando: 21144-27.mp3
✓ Descargado: archivos/21144-27.mp3


# **Ejercicio 2a:**

* #### **Comenten el por qué del modelo seleccionado para extracción del texto de los audios.**

* #### **Extraer el contenido de los audios en texto.**

* #### **Sugerencia:** pueden extraerlo en un formato de diccionario, clave:valor $→$ {audio01:fabula01, ...}

Para este punto comparamos los modelos"facebook/wav2vec2-large-xlsr-53-spanish"
 “jonatasgrosman/wav2vec2-large-xlsr-53-spanish", “facebook/wav2vec2-base-10k-voxpopuli-ft-es" y “openai/whisper-1”. Estos fueron los resultados obtenidos:

* Resultado modelo ‘Facebook/wav2vec2-large-xlsr-53-spanish’: fábulaesopo rcppaulinopaul el lobo y el cordero en el templo dándose cuenta de que era perseguido por un lobo un pequeño corderito decidió refugiarse en un templo cercano lo llamó lobo y le dijo que si el sacrificador lo encontraba allí adentro lo enmolaría a su dios mejor así replicó el cordero prefiero ser víctima para un dios a tener que perecer en tus colmillos si sin remedio vamos a ser sacrificados más nos vale que sea con el mayor honor fábula

* Resultado modelo ‘jonatasgrosman/wav2vec2-large-xlsr-53-spanish’: las fábulas de sopo grabado para libribox o hereg por paulino w w w  paulinoinfofábula número sesenta y uno el lobo y el cordero en el templodándo se cuenta de que era perseguido por un lobo un pequeño corderito decidió refugiarse en un templo cercano lo llamó lobo y le dijo que si el sacrificador lo encontraba allí adentro lo inmolaría a su dios mejor así replicó el cordero prefiero ser víctima para un dios a tener que perecer en tucolmillossi sin remedio vamos a ser sacrificados más nos vale que sea con el mayor honor fin de la fábulaesta es una grabación del dominio publico

* Resultado modelo ‘facebook/wav2vec2-base-10k-voxpopuli-ft-es': las faulas de sopo gravado para el libri vox punto o erg por paulino doble vé doble vé doble vé punto paulino puto info faula número sesenta y uno el lobo y el cordero en el templo tando se cuenta de que era perseguido por un lobo un pequeño cordedito desidió refugiarse en un templio cercano lo lamó lobo y le dijo que si el sacrificador lo encontraba allía dentro lo enmolaría a su dios mejor así eel cordero crefiero a ser víctima para un dios a tener que pereser en tus colmillos si sin remedio vamos a ser sacrificados más nos vale que sea con el mayor honor fin de la fabula esta es una gravación del dominio público

* Resultado modelo ‘openai/whisper-1’: Las fábulas de Sopo Grabado para LibriVox.org por Paulino www.paulino.info Fábula número 61 El lobo y el cordero en el templo Dándose cuenta de que era perseguido por un lobo, un pequeño corderito decidió refugiarse en un templo cercano. Lo llamó lobo y le dijo que si el sacrificador lo encontraba allí adentro, lo inmolaría a su dios. Mejor así, replicó el cordero, prefiero ser víctima para un dios a tener que perecer en tus colmillos. Si sin remedio vamos a ser sacrificados, más nos vale que sea con el mayor honor. Fin de la fábula Esta es una grabación del dominio público.

Al comparar las transcripciones de los cuatro modelos, el modelo ‘openai/whisper-1’ destaca claramente como el mejor. Su transcripción es la única que presenta una ortografía correcta, puntuación adecuada y segmentación de frases coherente, lo que facilita la comprensión lectora. Además, reproduce fielmente tanto la información principal del texto como los detalles contextuales, como los créditos y la mención al dominio público, sin errores notables en los nombres propios ni en los términos clave del relato. Esto contrasta con los otros modelos, que presentan errores de segmentación (“fábulaesopo rcppaulinopaul”), omisiones, palabras mal escritas (“gravado”, “templio”, “crefiero”) y confusiones en los nombres de dominio y enlaces.

El segundo modelo (‘jonatasgrosman/wav2vec2-large-xlsr-53-spanish’) también logra una transcripción bastante precisa en cuanto al contenido, pero carece de una puntuación adecuada y presenta algunas palabras pegadas o mal segmentadas, lo que dificulta la lectura. El tercer modelo (‘facebook/wav2vec2-base-10k-voxpopuli-ft-es’) muestra numerosos errores ortográficos y de reconocimiento de palabras, afectando la fidelidad del texto. El primer modelo (‘Facebook/wav2vec2-large-xlsr-53-spanish’) omite partes importantes y presenta errores graves de segmentación. En conclusión, ‘openai/whisper-1’ es el mejor modelo porque logra la mayor precisión, claridad y legibilidad en la transcripción, elementos fundamentales para una aplicación práctica y profesional de reconocimiento de voz.

In [ ]:
transcrip = {}

# Get all files in the folder
folder_path = '/content/archivos'
files_in_folder = os.listdir(folder_path)

# Iterate through each file
for file_name in files_in_folder:
    file_path = os.path.join(folder_path, file_name)
    # You can now do something with each file, for example, print its name
    print(f"Processing file: {file_path}")

    # Example: Check if it's a file (and not a subdirectory)
    if os.path.isfile(file_path):
        # Add your code here to process each file
        print(f"  This is a file.")

        # Example: Load and process audio if it's an mp3 file
        if file_path.endswith(".mp3"):


            # Ruta del archivo de audio en audio_path.
            # Abrimos el archivo en binario y extraemos el texto con whisper-1:
            with open(file_path, "rb") as audio_file:
                transcript = client.audio.transcriptions.create(model="whisper-1",
                                                                file=audio_file,
                                                                language="es"
                                                                )
                transcrip[file_name] = transcript.text

print("\nFinished processing all files in the folder.")

new_transcrip = {}
for key, value in transcrip.items():
  # Extract the last two digits before .mp3
  parts = key.split('-')
  if len(parts) > 1:
    number_part = parts[-1].split('.')[0]
    if number_part.isdigit() and len(number_part) >= 2:
      new_key = f"audio_{number_part[-2:]}"
      new_transcrip[new_key] = value
  else:
    # Handle cases where the key might not have the expected format
    new_transcrip[key] = value

transcrip = new_transcrip


Processing file: /content/archivos/21144-27.mp3
  This is a file.
Processing file: /content/archivos/21144-05.mp3
  This is a file.
Processing file: /content/archivos/21144-24.mp3
  This is a file.
Processing file: /content/archivos/21144-06.mp3
  This is a file.
Processing file: /content/archivos/21144-04.mp3
  This is a file.
Processing file: /content/archivos/21144-22.mp3
  This is a file.
Processing file: /content/archivos/21144-14.mp3
  This is a file.
Processing file: /content/archivos/21144-26.mp3
  This is a file.
Processing file: /content/archivos/21144-25.mp3
  This is a file.
Processing file: /content/archivos/21144-01.mp3
  This is a file.

Finished processing all files in the folder.


{'audio_27': 'Las fábulas de Esopo. Grabado para LibriVox.org por Elo Chito. Fábula número 87. El perro que perseguía al león. Un perro de casa se encontró con un león y partió en su persecución. Pero el león se volvió rugiendo y el perro, todo temorizado, retrocedió rápidamente por el mismo camino. Le vio una zorra y le dijo, ¡Perro infeliz! Primero perseguías al león y ya ni siquiera soportas sus surgidos. Cuando entres a una empresa, mantente siempre listo a afrontar imprevistos que no te imaginabas. Fin de la fábula. Esta grabación es del dominio público.',
 'audio_05': 'Las fábulas de Sopo. Grabado para LibriVox.org por Karen Savage. Fábula número 65. El lobo y el caballo. Pasaba un lobo por un sembrado de cebada, pero como no era comida de su gusto, la dejó y siguió su camino. Encontró al rato a un caballo y le llevó al campo comentándole la gran cantidad de cebada que había hallado, pero que en vez de comérsela a él, mejor se la había dejado porque le agradaba más oír el ruido d

In [ ]:
#Celda para guardar los resultados de la transcripción en un archivo JSON:

output_filename = "/content/drive/MyDrive/Colab Notebooks/Monterrey - Actividades y entregables/02 25 - NLP/Semana 7/transcriptions_openAI.json"

with open(output_filename, 'w', encoding='utf-8') as f:
    json.dump(transcrip, f, ensure_ascii=False, indent=4)

print(f"Transcription saved to {output_filename}")

Transcription saved to /content/drive/MyDrive/Colab Notebooks/Monterrey - Actividades y entregables/02 25 - NLP/Semana 7/transcriptions_openAI.json


In [ ]:
#Celda para cargar los resultados de la transcripción en un diccionario:

output_filename = "/content/drive/MyDrive/Colab Notebooks/Monterrey - Actividades y entregables/02 25 - NLP/Semana 7/transcriptions_openAI.json"

with open(output_filename, 'r', encoding='utf-8') as f:
    transcrip = json.load(f)

transcrip

{'audio_27': 'Las fábulas de Esopo. Grabado para LibriVox.org por Elo Chito. Fábula número 87. El perro que perseguía al león. Un perro de casa se encontró con un león y partió en su persecución. Pero el león se volvió rugiendo y el perro, todo temorizado, retrocedió rápidamente por el mismo camino. Le vio una zorra y le dijo, ¡Perro infeliz! Primero perseguías al león y ya ni siquiera soportas sus surgidos. Cuando entres a una empresa, mantente siempre listo a afrontar imprevistos que no te imaginabas. Fin de la fábula. Esta grabación es del dominio público.',
 'audio_05': 'Las fábulas de Sopo. Grabado para LibriVox.org por Karen Savage. Fábula número 65. El lobo y el caballo. Pasaba un lobo por un sembrado de cebada, pero como no era comida de su gusto, la dejó y siguió su camino. Encontró al rato a un caballo y le llevó al campo comentándole la gran cantidad de cebada que había hallado, pero que en vez de comérsela a él, mejor se la había dejado porque le agradaba más oír el ruido d

# **Ejercicio 2b:**

* #### **Eliminar el inicio y final comunes de los textos extraídos de cada fábula.**

* #### **Sugerencia:** Pueden guardar esta información en un archivo tipo JSON, para que al estar probando diferentes opciones en los ejercicios siguientes, puedan recuperar rápidamente la información de cada video/fábula.

In [ ]:
# Incluyan a continuación todas las celdas (de código o texto) que deseen...

def limpiar_transcripcion(texto):
    # Patrón para eliminar el inicio: todo hasta (e incluyendo) el número de la fábula y el punto inmediatamente posterior.
    texto = re.sub(r'^.*?\d{2}.', '', texto, flags=re.IGNORECASE|re.DOTALL)


    # Patrón para eliminar el final: desde "fin de la fábula" o similar hasta el final
    texto = re.sub(r'Fin de(?: la)? f[aá]bula.*$', '', texto, flags=re.IGNORECASE|re.DOTALL)
    # Limpiar espacios extra
    texto = texto.strip()

    return texto

# Aplicar limpieza a cada transcripción
transcripciones_limpias = {
    k: limpiar_transcripcion(v)
    for k, v in transcrip.items()
}

# Imprimir resultados
for clave, texto in transcripciones_limpias.items():
    print(f"{clave}:\n{texto}\n{'-'*60}")

audio_27:
El perro que perseguía al león. Un perro de casa se encontró con un león y partió en su persecución. Pero el león se volvió rugiendo y el perro, todo temorizado, retrocedió rápidamente por el mismo camino. Le vio una zorra y le dijo, ¡Perro infeliz! Primero perseguías al león y ya ni siquiera soportas sus surgidos. Cuando entres a una empresa, mantente siempre listo a afrontar imprevistos que no te imaginabas.
------------------------------------------------------------
audio_05:
El lobo y el caballo. Pasaba un lobo por un sembrado de cebada, pero como no era comida de su gusto, la dejó y siguió su camino. Encontró al rato a un caballo y le llevó al campo comentándole la gran cantidad de cebada que había hallado, pero que en vez de comérsela a él, mejor se la había dejado porque le agradaba más oír el ruido de sus dientes al masticarla. Pero el caballo le repuso. Amigo, si los lobos comieran cebada, no hubieras preferido complacer a tus oídos sino a tu estómago. A todo malvad

# **Ejercicio 3:**

* #### **Apliquen el proceso de limpieza que consideren adecuado.**

* #### **Justifiquen los pasos de limpieza utilizados. Tomen en cuenta que el texto extraído de cada fábula es relativamente pequeño.**

* #### **En caso de que decidan no aplicar esta etapa de limpieza, deberán justificarlo.**

Antes de aplicar el modelo LDA (Latent Dirichlet Allocation) para la extracción de temas en textos, es fundamental realizar un proceso de limpieza lingüística. Este paso mejora significativamente la calidad del análisis, ya que LDA es sensible al ruido textual. En la función limpiar_texto, primero se convierten todos los caracteres a minúsculas para evitar duplicidad de palabras con distintas capitalizaciones. Luego, se eliminan los números y signos de puntuación, ya que no aportan información semántica relevante al modelo.

Posteriormente, se eliminan las palabras vacías (stopwords) del español, que son términos comunes sin carga temática (como “el”, “y”, “de”), y se descartan palabras demasiado cortas, que suelen ser poco informativas. Este preprocesamiento asegura que el modelo LDA se concentre en las palabras más representativas y temáticamente significativas del texto, facilitando la identificación de tópicos coherentes.

In [ ]:
# Incluyan a continuación todas las celdas (de código o texto) que deseen...

def limpiar_texto(texto):
    texto = texto.lower()
    texto = re.sub(r'\d+', '', texto)  # eliminar números
    texto = re.sub(r'[' + string.punctuation + '¿¡]', '', texto)  # quitar puntuación
    palabras = texto.split()
    palabras_limpias = [p for p in palabras if p not in stop_words and len(p) > 2]
    return ' '.join(palabras_limpias)


# **Ejercicio 4:**

En este fragmento de código se aplica el modelo LDA a cada fábula de forma individual para extraer sus palabras clave más representativas. Primero, se limpia el texto y se vectoriza usando el modelo de bolsa de palabras. Luego, se entrena un modelo LDA configurado para identificar un único tópico por fábula. A partir del tópico identificado, se extraen las 20 palabras más relevantes (según su peso en el modelo), que se consideran las palabras clave del texto. Este proceso se repite para cada fábula, almacenando los resultados en un diccionario para su posterior análisis o visualización.

In [ ]:
# Incluyan a continuación todas las celdas (de código o texto) que deseen...

n_top_words = 20

palabras_clave = {}

for nombre, texto_original in transcripciones_limpias.items():
    texto = limpiar_texto(texto_original)

    # Vectorización (bolsa de palabras)
    vectorizer = CountVectorizer()
    X = vectorizer.fit_transform([texto])  # Solo una fábula

    if X.shape[1] < n_top_words:
        print(f"{nombre}: ⚠️ No hay suficientes palabras para extraer {n_top_words}")
        continue

    # LDA con 1 solo tópico
    lda = LatentDirichletAllocation(n_components=1, max_iter=10, random_state=42)
    lda.fit(X)

    # Palabras clave del tema principal
    palabras = vectorizer.get_feature_names_out()
    tema = lda.components_[0]
    indices_top = tema.argsort()[-n_top_words:][::-1]
    top_words = [palabras[i] for i in indices_top]
    palabras_clave[nombre] = top_words

    # Mostrar resultado
    print(f"\n {nombre}:")
    print("Palabras clave:", ', '.join(top_words))


🔹 audio_27:
Palabras clave: perro, león, zorra, temorizado, surgidos, volvió, vio, siempre, rápidamente, rugiendo, retrocedió, primero, perseguías, soportas, siquiera, persecución, perseguía, mismo, partió, mantente

🔹 audio_05:
Palabras clave: caballo, cebada, lobo, sembrado, ruido, repuso, rato, preferido, vez, sino, siguió, oír, parezca, pasaba, oídos, malvado, lobos, masticarla, mejor, gusto

🔹 audio_24:
Palabras clave: reflejo, río, perro, ajeno, propio, trozo, pedazo, carne, sabroso, solo, soltó, supuesto, vio, verdadero, quedó, realidad, pues, perder, nunca, mayor

🔹 audio_06:
Palabras clave: lobo, asno, ley, tener, rey, repartiese, repártelo, propias, primero, poder, pusiera, partes, orejas, moviendo, ordenando, manera, magnífica, vez, épocas, llévalo

🔹 audio_04:
Palabras clave: lobo, hueso, paga, garganta, grulla, cabeza, boca, pidió, todas, suficiente, salvo, sacado, sacando, situación, sano, salvara, salva, sana, traficantes, mucha

🔹 audio_22:
Palabras clave: luego, perro

# **Ejercicio 5a y 5b:**

* #### **5a: Mediante el LLM que hayan seleccionado, generar un único enunciado que describa o resuma cada fábula.**

* #### **5b: Mediante el LLM que hayan seleccionado, generar tres posibles enunciados diferentes relacionados con la historia de la fábula.**

* #### **Sugerencia:** En realidad los dos incisos a y b se pueden obtener con un solo prompt que solicite la información y el formato correspondiente para cada una de estas partes. Por ejemplo, para cada fábula la salida puede ser un primer enunciado genérico que resume o describe dicha temática; seguido de tres enunciados, cada uno hablando sobre una situación o parte diferente de la fábula.

Este fragmento utiliza un modelo de lenguaje (LLM) para generar, a partir de las palabras clave de cada fábula, un resumen y tres posibles subtemas. Para ello, se construye un prompt que describe la tarea a realizar usando la lista de palabras clave asociada a cada fábula. Luego, el modelo genera una respuesta con un enunciado que resume el contenido de la fábula y tres frases que representan posibles subtemas relacionados. Los resultados se almacenan en un diccionario para su posterior consulta o análisis.

In [ ]:
# Incluyan a continuación todas las celdas (de código o texto) que deseen...

def generar_resumen_y_subtemas(palabras_clave, modelo="gpt-4"):
    resultados = {}
    for nombre, lista in palabras_clave.items():
        prompt = (
            f"Con base en esta lista de palabras clave: {', '.join(lista)}\n"
            "1. Genera un enunciado que resuma una fábula donde estas palabras aparezcan.\n"
            "2. Luego, genera tres posibles subtemas relacionados (como frases).\n"
        )
        response = client.chat.completions.create(
            model=modelo,
            messages=[{"role": "user", "content": prompt}],
            temperature=0.7
        )
        texto = response.choices[0].message.content
        resultados[nombre] = texto
    return resultados

resultados = generar_resumen_y_subtemas(palabras_clave)
resultados

{'audio_27': '1. En la fábula, el perro siempre perseguía a la zorra, pero cuando vio al león rugiendo, se sintió temorizado y rápidamente retrocedió. El león, surgidos de la nada, se volvió hacia el perro que antes perseguía sin temor a otros animales. Pero ahora, ni siquiera podía soportar la persecución del rey de las bestias. El perro, aprendiendo su lección, partió con el consejo de mantenerse siempre en su lugar y no molestar a los demás.\n\n2. Subtemas:\n   - El papel de la intimidación y el miedo en las relaciones de poder.\n   - La importancia de conocer y mantener nuestro lugar en la sociedad.\n   - Las consecuencias de molestar o intimidar a los demás.',
 'audio_05': '1. En la fábula, un malvado lobo intenta engañar a un caballo para que salga de su sembrado de cebada, su lugar preferido, y se convierta en su presa. El caballo, a pesar de oír los ruidos y las falsas promesas del lobo, decide seguir en su sembrado. Con el paso del tiempo, el lobo se repuso y volvió a atacar, 

# **Ejercicio 6:**

* #### **Incluyan sus conclusiones de la actividad audio-a-texto:**



Los resultados de esta actividad demuestran con claridad el rendimiento superior del modelo openai/whisper-1 en tareas de transcripción automática. Su capacidad para preservar la ortografía, puntuación y segmentación del discurso lo posiciona como una herramienta altamente confiable para aplicaciones profesionales en procesamiento de audio. A diferencia de otros modelos evaluados, Whisper-1 mantiene la fidelidad semántica del mensaje original, incluso en los detalles contextuales más sutiles. Esto es especialmente relevante cuando se busca preservar la intención narrativa de textos como fábulas, donde los matices lingüísticos y morales deben conservarse con precisión.

Además, el análisis temático mediante LDA y la posterior interpretación generada con un modelo de lenguaje permitió extraer el núcleo narrativo de cada audio y elaborar subtemas significativos. Estos subtemas revelan patrones morales y estructurales recurrentes en las fábulas, como la lucha entre fuerza y astucia, el aprendizaje a través del error, o la justicia en contextos simbólicos. Esta combinación de herramientas no solo facilita la clasificación y comprensión automatizada de contenidos orales, sino que también abre posibilidades para el desarrollo de sistemas educativos y de análisis literario apoyados por inteligencia artificial.

# **Fin de la actividad LDA y LMM: audio-a-texto**